In [40]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten

# --- CONFIGURACIÓN Y CONSTANTES ---
FILE_NAME = "cuadrantes_negocios.csv"
NUM_CUADRANTES = 78
EMBEDDING_DIM = 10 # Dimensión del vector de Embedding para el cuadrante

# --- 1. CARGA, TRANSFORMACIÓN (ANCHO A LARGO) Y GENERACIÓN DE FEATURES ---

def load_and_prepare_data(file_name):
    # Cargar el archivo
    try:
        df_wide = pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{file_name}'.")
        raise

    # 1.1. Limpieza inicial y Renombre
    df_wide.rename(columns={'CUADRANTE': 'CUADRANTE_ID', 'POBLACION': 'POBLACION_BASE'}, inplace=True)
    
    id_vars = ['CUADRANTE_ID', 'POBLACION_BASE']
    value_vars = [col for col in df_wide.columns if col not in id_vars]

    # 1.2. Transformación de Ancho a Largo (Melt)
    df_long = df_wide.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='MES_AÑO_STR',
        value_name='ROBOS_TOTAL'
    ).copy()

    # 1.3. Parsear la fecha
    def parse_date(date_str):
        parts = date_str.split(' ')
        # Asumiendo el formato 'ROBOS A NEGOCIOS MES 1 2015'
        mes = parts[4] 
        año = parts[5] 
        return f'{año}-{mes}-01'

    df_long['FECHA'] = df_long['MES_AÑO_STR'].apply(parse_date)
    df_long['FECHA'] = pd.to_datetime(df_long['FECHA'])
    df_long.sort_values(by=['CUADRANTE_ID', 'FECHA'], inplace=True)
    df_long.drop(columns=['MES_AÑO_STR'], inplace=True)

    # 1.4. Generación de Características Temporales
    df_long['AÑO'] = df_long['FECHA'].dt.year
    df_long['MES'] = df_long['FECHA'].dt.month

    # 1.5. Generación de Características Históricas (Retardos)
    df_long['ROBOS_t-1'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(1)
    df_long['ROBOS_t-12'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(12)
    
    # Media Móvil de 3 meses (excluyendo el mes actual)
    df_long['MEDIA_MOVIL_3M'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)
    
    # Eliminar filas con NaN en los retardos (primeros 12 meses)
    df_model_input = df_long.dropna(subset=['ROBOS_t-12']).copy()
    
    return df_model_input

# Cargar y preparar los datos
df_model_input = load_and_prepare_data(FILE_NAME)


# --- 2. VERIFICACIÓN, LIMPIEZA DE TIPOS, NORMALIZACIÓN Y CODIFICACIÓN ---

# 2.1. Verificación y Corrección de Tipos (SOLUCIÓN al ValueError)
numerical_and_target_cols = ['POBLACION_BASE', 'ROBOS_t-1', 'ROBOS_t-12', 'MEDIA_MOVIL_3M', 'ROBOS_TOTAL']
for col in numerical_and_target_cols:
    df_model_input[col] = pd.to_numeric(df_model_input[col], errors='coerce')

df_model_input.dropna(subset=numerical_and_target_cols, inplace=True)
df_model_input['CUADRANTE_ID'] = df_model_input['CUADRANTE_ID'].astype(np.int32)
df_model_input['MES'] = df_model_input['MES'].astype(np.int32)

print(f"Total de filas listas para modelar: {len(df_model_input)}")

# 2.2. Normalización Min-Max (Guardar escaler del target)
scaler = MinMaxScaler()
scaler_robos_total = MinMaxScaler() 

# Normalizar todas las variables numéricas y el target
df_model_input[numerical_and_target_cols] = scaler.fit_transform(df_model_input[numerical_and_target_cols])
scaler_robos_total.fit(df_model_input[['ROBOS_TOTAL']]) # El target se escala de nuevo solo para guardar su objeto scaler

# 2.3. Codificación One-Hot para el MES
df_processed = pd.get_dummies(df_model_input, columns=['MES'], prefix='MES')


# --- 3. DIVISIÓN DE DATOS (Secuencial) Y PREPARACIÓN DE ARRAYS ---

FECHA_CORTE_VAL = pd.to_datetime('2023-01-01')
FECHA_CORTE_TEST = pd.to_datetime('2024-01-01')

# División
df_train = df_processed[df_processed['FECHA'] < FECHA_CORTE_VAL]
df_val = df_processed[(df_processed['FECHA'] >= FECHA_CORTE_VAL) & (df_processed['FECHA'] < FECHA_CORTE_TEST)]
df_test = df_processed[df_processed['FECHA'] >= FECHA_CORTE_TEST]

# Definición de Features para la rama numérica
features = [
    'POBLACION_BASE', 
    'ROBOS_t-1', 
    'ROBOS_t-12', 
    'MEDIA_MOVIL_3M'
] + [col for col in df_processed.columns if 'MES_' in col]


# Conversión a Arrays de NumPy con tipo explícito (SOLUCIÓN FINAL al error)
# 1. Entradas para el Embedding (CUADRANTE_ID) - int32
X_cuadrante_train = df_train['CUADRANTE_ID'].values.astype(np.int32)
X_cuadrante_val = df_val['CUADRANTE_ID'].values.astype(np.int32)

# 2. Entradas para las Características Numéricas - float32
X_num_train = df_train[features].values.astype(np.float32)
X_num_val = df_val[features].values.astype(np.float32)

# 3. Salidas (Objetivo) - float32
Y_train = df_train['ROBOS_TOTAL'].values.astype(np.float32)
Y_val = df_val['ROBOS_TOTAL'].values.astype(np.float32)


# --- 4. CONSTRUCCIÓN Y ENTRENAMIENTO DEL MODELO DE RED NEURONAL ---

# 4.1. Definición de la Arquitectura
def build_model(num_cuadrantes, embedding_dim, num_feat_numericas):
    
    # 1. Rama Embedding
    input_cuadrante = Input(shape=(1,), name='input_cuadrante')
    embedding_layer = Embedding(
        input_dim=num_cuadrantes + 1, 
        output_dim=embedding_dim,
        input_length=1,
        name='embedding_cuadrante'
    )(input_cuadrante)
    flatten_embedding = Flatten()(embedding_layer)

    # 2. Rama Numérica
    input_numerico = Input(shape=(num_feat_numericas,), name='input_numerico')

    # 3. Concatenación y Capas Profundas
    combined = Concatenate()([flatten_embedding, input_numerico])
    
    # Capas Ocultas (Dense)
    dense_1 = Dense(64, activation='relu')(combined)
    dense_2 = Dense(32, activation='relu')(dense_1)
    
    # 4. Capa de Salida (Regresión Lineal)
    output_layer = Dense(1, activation='linear', name='prediccion_final')(dense_2)

    model = Model(inputs=[input_cuadrante, input_numerico], outputs=output_layer)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

model = build_model(NUM_CUADRANTES, EMBEDDING_DIM, X_num_train.shape[1])
# print(model.summary())

# 4.2. Entrenamiento
print("\n--- COMENZANDO ENTRENAMIENTO DE LA RED NEURONAL ---")
history = model.fit(
    x=[X_cuadrante_train, X_num_train],
    y=Y_train,
    validation_data=([X_cuadrante_val, X_num_val], Y_val),
    epochs=50, 
    batch_size=32,
    verbose=1
)
print("✅ Entrenamiento del Modelo de Red Neuronal Finalizado.")


# --- 5. PREPARACIÓN Y PREDICCIÓN FINAL (Ejemplo para Enero 2024) ---

# Usamos el set de prueba para demostrar la predicción
X_cuadrante_test = df_test['CUADRANTE_ID'].values.astype(np.int32)
X_num_test = df_test[features].values.astype(np.float32)

# 5.1. Predicción en escala normalizada
y_pred_scaled = model.predict([X_cuadrante_test, X_num_test])

# 5.2. Desnormalizar la predicción
# Se crea un array 2D para inverse_transform
y_pred_denormalized = scaler_robos_total.inverse_transform(y_pred_scaled)

# 5.3. Formatear las 78 Salidas
df_results = pd.DataFrame({
    'CUADRANTE_ID': df_test['CUADRANTE_ID'].values,
    'FECHA': df_test['FECHA'].values,
    'PREDICCION_ROBOS_REAL': y_pred_denormalized.flatten()
})



Total de filas listas para modelar: 8424

--- COMENZANDO ENTRENAMIENTO DE LA RED NEURONAL ---
Epoch 1/50


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - mae: 0.0714 - val_loss: 0.0074 - val_mae: 0.0641
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - mae: 0.0714 - val_loss: 0.0074 - val_mae: 0.0641
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0651 - val_loss: 0.0072 - val_mae: 0.0622
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0651 - val_loss: 0.0072 - val_mae: 0.0622
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0637 - val_loss: 0.0077 - val_mae: 0.0654
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0637 - val_loss: 0.0077 - val_mae: 0.0654
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0628 - val_loss: 0.0068 - val_mae: 0.0604
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0628 - val_loss: 0.0068 - val_mae: 0.0604
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079 

In [41]:
# Filtrar un mes específico y ordenar (ej: Enero 2024)
MES_A_EVALUAR = 1
prediccion_enero = df_results[df_results['FECHA'].dt.month == MES_A_EVALUAR].copy()
prediccion_enero['PREDICCION_ROBOS_REAL'] = prediccion_enero['PREDICCION_ROBOS_REAL'].round(0).astype(int)
prediccion_enero = prediccion_enero.sort_values(by='PREDICCION_ROBOS_REAL', ascending=False)


print(f"\n--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---")
print(prediccion_enero[['CUADRANTE_ID', 'PREDICCION_ROBOS_REAL']].head(10))


--- TOP 10 CUADRANTES CON MAYOR INCIDENCIA (Predicción para Enero 2024) ---
     CUADRANTE_ID  PREDICCION_ROBOS_REAL
0               1                      0
12              2                      0
24              3                      0
36              4                      0
48              5                      0
60              6                      0
72              7                      0
84              8                      0
96              9                      0
108            10                      0


In [ ]:
import pandas as pd
import numpy as np

# --- 1. Carga y Transformación de Formato Ancho a Largo ---

file_name = "cuadrantes_negocios.csv"
df_wide = pd.read_csv(file_name)


# Renombrar 'CUADRANTE' a 'CUADRANTE_ID' y 'POBLACION'
df_wide.rename(columns={'CUADRANTE': 'CUADRANTE_ID', 'POBLACION': 'POBLACION_BASE'}, inplace=True)

# Identificar las columnas de robos (todas las columnas excepto 'CUADRANTE_ID' y 'POBLACION_BASE')
id_vars = ['CUADRANTE_ID', 'POBLACION_BASE']
value_vars = [col for col in df_wide.columns if col not in id_vars]

# Transformación de Formato Ancho a Largo (Melting)
# Esto crea una fila por cada combinación de Cuadrante y Mes/Año
df_long = df_wide.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='MES_AÑO_STR', # Columna temporal que contiene el mes y el año
    value_name='ROBOS_TOTAL' # La variable objetivo
).copy()

# 1.1. Parsear la columna MES_AÑO_STR para crear la columna de FECHA
# Ejemplo: 'ROBOS A NEGOCIOS MES 1 2015' -> Enero 2015
def parse_date(date_str):
    # Separar la cadena por espacios
    parts = date_str.split(' ')
    mes = parts[4] # Ej: '1'
    año = parts[5] # Ej: '2015'
    return f'{año}-{mes}-01'

df_long['FECHA'] = df_long['MES_AÑO_STR'].apply(parse_date)
df_long['FECHA'] = pd.to_datetime(df_long['FECHA'])
df_long.sort_values(by=['CUADRANTE_ID', 'FECHA'], inplace=True)
df_long.drop(columns=['MES_AÑO_STR'], inplace=True)


# --- 2. Generación de Características Temporales ---

df_long['AÑO'] = df_long['FECHA'].dt.year
df_long['MES'] = df_long['FECHA'].dt.month
# df_long['DIA_DEL_MES'] = df_long['FECHA'].dt.day # Si fuera relevante, pero para mensual, el mes ya lo captura

# --- 3. Generación de Características Históricas (Retardos) ---

# Agrupar por CUADRANTE_ID para asegurar que el retardo sea solo dentro de esa serie de tiempo.
df_long['ROBOS_t-1'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(1)
df_long['ROBOS_t-12'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].shift(12)

# Generación de la Media Móvil de 3 meses (excluyendo el mes actual)
# Usamos un Rolling Window de 3 periodos y lo shift(1) para aplicarlo al mes siguiente.
df_long['MEDIA_MOVIL_3M'] = df_long.groupby('CUADRANTE_ID')['ROBOS_TOTAL'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1)


# --- 4. Limpieza Final para el Modelo ---

# Eliminar las filas que tienen valores NaN en los retardos (los primeros 12 meses de cada cuadrante)
# Estos no tienen suficiente historial para las variables t-12.
df_model_input = df_long.dropna(subset=['ROBOS_t-12']).copy()

# Seleccionar solo las columnas que servirán como entradas (X) y la salida (Y)
# NOTA: La POBLACION_BASE es estática. Otras variables como la violencia se añadirían aquí.
columnas_finales = [
    'CUADRANTE_ID',
    'FECHA',
    'AÑO',
    'MES',
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    'ROBOS_TOTAL' # Esta será la variable objetivo (Y)
]

df_model_input = df_model_input[columnas_finales]

print("--- Vista Preliminar de la Tabla de Entradas (Formato Largo) ---")
print(f"Total de filas generadas para el modelo: {len(df_model_input)}")
print("Ejemplo de las entradas para el CUADRANTE 1:")
print(df_model_input[df_model_input['CUADRANTE_ID'] == 1].head(15))

In [ ]:
# exportar df_model_input a un archivo csv
df_model_input.to_csv("ficosec_model_input.csv", index=False)

In [ ]:
# exportar df_model_input a un archivo xlsx
df_model_input.to_excel("ficosec_model_input.xlsx", index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Suponiendo que 'df_model_input' es el DataFrame generado en el paso anterior

# 1. Identificar las columnas numéricas que necesitan escalado
features_to_scale = [
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    # La variable objetivo (Y) también debe escalarse
    'ROBOS_TOTAL' 
]

# 2. Inicializar el escalador
scaler = MinMaxScaler()

# 3. Aplicar el escalador a las columnas seleccionadas
df_model_input[features_to_scale] = scaler.fit_transform(df_model_input[features_to_scale])

# Guardar los escaladores si planeas desnormalizar las predicciones más tarde
# Por ejemplo, el scaler de ROBOS_TOTAL es crucial para llevar la predicción de vuelta a conteos reales.
scaler_robos_total = MinMaxScaler()
scaler_robos_total.fit(df_model_input[['ROBOS_TOTAL']]) # Se debe re-entrenar solo en el target si quieres desnormalizar fácilmente

print("✅ Normalización Min-Max aplicada a las variables numéricas.")
print(df_model_input[features_to_scale].head())

In [ ]:
# --- A. Codificación One-Hot para el MES ---

# Crear variables dummy para la columna 'MES'
df_processed = pd.get_dummies(df_model_input, columns=['MES'], prefix='MES')

# El CUADRANTE_ID se mantiene como está para el Embedding (se usará directamente en Keras/TensorFlow)

print("\n✅ Codificación One-Hot aplicada al MES.")
print(df_processed[[col for col in df_processed.columns if 'MES_' in col]].head())

In [ ]:
# 1. Definir las fechas de corte
FECHA_CORTE_VAL = pd.to_datetime('2023-01-01')
FECHA_CORTE_TEST = pd.to_datetime('2024-01-01')

# 2. Dividir los datos
df_train = df_processed[df_processed['FECHA'] < FECHA_CORTE_VAL].copy()
df_val = df_processed[(df_processed['FECHA'] >= FECHA_CORTE_VAL) & (df_processed['FECHA'] < FECHA_CORTE_TEST)].copy()
df_test = df_processed[df_processed['FECHA'] >= FECHA_CORTE_TEST].copy()

# 3. Definición de Entradas (X) y Salidas (Y)
features = [
    'POBLACION_BASE', 
    'ROBOS_t-1', 
    'ROBOS_t-12', 
    'MEDIA_MOVIL_3M'
] + [col for col in df_processed.columns if 'MES_' in col]

X_train = df_train[features]
Y_train = df_train['ROBOS_TOTAL']

X_val = df_val[features]
Y_val = df_val['ROBOS_TOTAL']

X_test = df_test[features]
Y_test = df_test['ROBOS_TOTAL']

# El CUADRANTE_ID se manejará como una entrada separada en el modelo de Keras/TensorFlow
train_cuadrante_id = df_train['CUADRANTE_ID']
val_cuadrante_id = df_val['CUADRANTE_ID']
test_cuadrante_id = df_test['CUADRANTE_ID']

print(f"\n✅ División de datos completada:")
print(f"  Entrenamiento (Train): {len(X_train)} filas")
print(f"  Validación (Validation): {len(X_val)} filas")
print(f"  Prueba (Test): {len(X_test)} filas")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten

# --- CONSTANTES ---
NUM_CUADRANTES = df_model_input['CUADRANTE_ID'].nunique() # Debería ser 78
# La dimensión del embedding debe ser menor que el número de categorías. 
# Una regla empírica simple es tomar la raíz cuadrada del número de categorías.
EMBEDDING_DIM = int(np.sqrt(NUM_CUADRANTES)) # Ej: sqrt(78) ~ 8 o 9. Usemos 10.
EMBEDDING_DIM = 10
NUM_FEAT_NUMERICAS = X_train.shape[1] 

# --- CONVERSIÓN A ARRAYS DE NUMPY ---

# 1. Entradas para el Embedding (CUADRANTE_ID)
# Keras requiere que los IDs empiecen en 0 o 1. Usaremos 1 a N.
# NOTA: Los DataFrames 'train_cuadrante_id', 'val_cuadrante_id', 'test_cuadrante_id'
# ya están preparados para esto (valores de 1 a 78).
X_cuadrante_train = train_cuadrante_id.values
X_cuadrante_val = val_cuadrante_id.values
X_cuadrante_test = test_cuadrante_id.values

# 2. Entradas para las Características Numéricas (Robos t-1, Mes_1, etc.)
X_num_train = X_train.values
X_num_val = X_val.values
X_num_test = X_test.values

# 3. Salidas (Objetivo)
Y_train = Y_train.values
Y_val = Y_val.values
Y_test = Y_test.values

print(f"Número de Cuadrantes (Categorías): {NUM_CUADRANTES}")
print(f"Dimensión del Vector de Embedding: {EMBEDDING_DIM}")
print(f"Número de Características Numéricas/One-Hot: {NUM_FEAT_NUMERICAS}")

In [ ]:
# --- 1. Rama de Entrada para el CUADRANTE_ID (Embedding Layer) ---

# Input: Un array de números enteros (IDs de 1 a 78)
input_cuadrante = Input(shape=(1,), name='input_cuadrante')

# Embedding: Convierte cada ID de cuadrante en un vector denso (ej: 78x10)
# input_dim: Rango de los IDs (78 cuadrantes + 1 por convención)
# output_dim: La dimensión de nuestro vector de embedding (10)
embedding_layer = Embedding(
    input_dim=NUM_CUADRANTES + 1,
    output_dim=EMBEDDING_DIM,
    input_length=1,
    name='embedding_cuadrante'
)(input_cuadrante)

# Aplanar el Embedding (de (None, 1, 10) a (None, 10))
flatten_embedding = Flatten()(embedding_layer)


# --- 2. Rama de Entrada para las Características Numéricas ---

# Input: Un array con todas las características (Robos t-1, Mes_1, Población, etc.)
input_numerico = Input(shape=(NUM_FEAT_NUMERICAS,), name='input_numerico')


# --- 3. Combinación de Ramas (Merging) ---

# Concatenar el vector de embedding y las características numéricas
combined = Concatenate()([flatten_embedding, input_numerico])

# Capas Densa de la Red Neuronal (Deep Layers)
# Se utilizan capas profundas para aprender las interacciones complejas.
dense_1 = Dense(64, activation='relu')(combined)
dense_2 = Dense(32, activation='relu')(dense_1)

# Capa de Salida
# Output: Un único nodo que predice el valor escalado de ROBOS_TOTAL (Y).
output_layer = Dense(1, activation='linear', name='prediccion_final')(dense_2)


# --- 4. Crear el Modelo Final ---

model = Model(inputs=[input_cuadrante, input_numerico], outputs=output_layer)

# Compilación: Usamos 'mean_squared_error' (MSE) para regresión y Adam como optimizador.
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(model.summary())

In [ ]:
# --- CÓDIGO DE VERIFICACIÓN Y CORRECCIÓN DE TIPOS (Solución al ValueError) ---

# 1. Identificar todas las columnas que deberían ser numéricas (inputs y targets)
numerical_and_target_cols = [
    'POBLACION_BASE',
    'ROBOS_t-1',
    'ROBOS_t-12',
    'MEDIA_MOVIL_3M',
    'ROBOS_TOTAL'
]

# 2. Forzar la conversión de estas columnas a numérico (float)
# El argumento 'coerce' convierte cualquier valor no numérico (texto) a NaN.
for col in numerical_and_target_cols:
    df_model_input[col] = pd.to_numeric(df_model_input[col], errors='coerce')

# 3. Eliminar o Rellenar los valores NaN que se generaron
# Es más seguro eliminar las filas donde esto ocurre, aunque deberían ser pocas.
# Si el error ocurrió en una fila crítica (ej. ROBOS_TOTAL), la fila es inutilizable.
filas_antes = len(df_model_input)
df_model_input.dropna(subset=numerical_and_target_cols, inplace=True)
filas_despues = len(df_model_input)

if filas_antes != filas_despues:
    print(f"⚠️ Alerta: Se eliminaron {filas_antes - filas_despues} filas debido a valores no numéricos ocultos.")

# 4. Asegurar que las columnas de ID sean Enteros (para el Embedding)
# El Embedding Layer requiere IDs enteros.
df_model_input['CUADRANTE_ID'] = df_model_input['CUADRANTE_ID'].astype(np.int32)
df_model_input['MES'] = df_model_input['MES'].astype(np.int32) 

print("\n✅ Verificación de tipos de datos completada. Todos los inputs/outputs numéricos están limpios.")

In [ ]:
print(X_num_train.dtype)

In [ ]:
# convertir X_num_train a float32
X_num_train = X_num_train.astype(np.int32)
X_num_test = X_num_test.astype(np.int32)

In [ ]:
print(X_cuadrante_train.dtype)

In [ ]:
# convertir X_cuadrante_train a float64
X_cuadrante_train = X_cuadrante_train.astype(np.float64)
X_cuadrante_test = X_cuadrante_test.astype(np.float64)

In [ ]:
# Configuración del entrenamiento
history = model.fit(
    x=[X_cuadrante_train, X_num_train],
    y=Y_train,
    validation_data=([X_cuadrante_val, X_num_val], Y_val),
    epochs=50, # Puedes empezar con 50 o 100 épocas
    batch_size=32,
    verbose=1
)

print("\n✅ Entrenamiento del Modelo de Red Neuronal Finalizado.")